Сформировать отчёт с информацией о 10 наиболее популярных языках программирования по итогам года за период с 2010 по 2020 годы. Отчёт будет отражать динамику изменения популярности языков программирования и представлять собой набор таблиц "топ-10" для каждого года.

Получившийся отчёт сохранить в формате Apache Parquet.

In [1]:
!pip3 install pyspark==3.5.0


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install wget


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
import sys
import pyspark.sql.functions as F
from pyspark.sql import Row
from pyspark.sql import SparkSession
import wget
import requests
from datetime import datetime
from pyspark.sql.functions import year, col

In [4]:
hadoop_dir = "C:\\hadoop\\bin"
os.makedirs(hadoop_dir, exist_ok=True)

winutils_url = "https://github.com/cdarlint/winutils/raw/master/hadoop-3.3.6/bin/winutils.exe"
wget.download(winutils_url, out=hadoop_dir)

100% [............................................................................] 119296 / 119296

'C:\\hadoop\\bin/winutils (1).exe'

In [5]:
os.environ['HADOOP_HOME'] = "C:\\hadoop"
os.environ['PATH'] = f"{os.environ['HADOOP_HOME']}\\bin;{os.environ['PATH']}"

In [6]:
# Проверяем доступ к winutils
if os.path.exists(os.path.join(hadoop_dir, "winutils.exe")):
    print("\nwinutils успешно установлен!")
else:
    print("\nНе удалось установить winutils!")


winutils успешно установлен!


In [7]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable


spark = SparkSession.builder \
    .appName("XMLParser") \
    .config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.13.0") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()

print("Версия Spark:", spark.version)
spark

Версия Spark: 3.5.0


In [8]:
url = "https://git.ai.ssau.ru/tk/big_data/raw/branch/master/data/posts_sample.xml"
file_path = "posts_sample.xml"

if not os.path.exists(file_path):
    response = requests.get(url)
    with open(file_path, 'wb') as f:
        f.write(response.content)
    print("Файл успешно скачан")
else:
    print("Файл уже существует")


absolute_path = os.path.abspath(file_path)
print(f"Абсолютный путь к файлу: {absolute_path}")

try:
    postsData = spark.read.format('xml') \
        .option('rowTag', 'row') \
        .option("timestampFormat", 'y/M/d H:m:s') \
        .load(absolute_path)
    
    print("Данные успешно загружены!")
except Exception as e:
    print(f"Ошибка при загрузке данных: {str(e)}")

Файл уже существует
Абсолютный путь к файлу: C:\Users\lena\Documents\Университет с 05-23\8 семестр\Биг дата\Lab_2\posts_sample.xml
Данные успешно загружены!


In [9]:
print("Схема")
postsData.printSchema()

print("Первые 5 элементов")
postsData.show(n = 5)

print("Описание датасета")
postsData.describe().show()

print("Количество элементов")
print(postsData.count())

Схема
root
 |-- _AcceptedAnswerId: long (nullable = true)
 |-- _AnswerCount: long (nullable = true)
 |-- _Body: string (nullable = true)
 |-- _ClosedDate: timestamp (nullable = true)
 |-- _CommentCount: long (nullable = true)
 |-- _CommunityOwnedDate: timestamp (nullable = true)
 |-- _CreationDate: timestamp (nullable = true)
 |-- _FavoriteCount: long (nullable = true)
 |-- _Id: long (nullable = true)
 |-- _LastActivityDate: timestamp (nullable = true)
 |-- _LastEditDate: timestamp (nullable = true)
 |-- _LastEditorDisplayName: string (nullable = true)
 |-- _LastEditorUserId: long (nullable = true)
 |-- _OwnerDisplayName: string (nullable = true)
 |-- _OwnerUserId: long (nullable = true)
 |-- _ParentId: long (nullable = true)
 |-- _PostTypeId: long (nullable = true)
 |-- _Score: long (nullable = true)
 |-- _Tags: string (nullable = true)
 |-- _Title: string (nullable = true)
 |-- _ViewCount: long (nullable = true)

Первые 5 элементов
+-----------------+------------+--------------------

In [10]:
# Ищем с 1 января 2010 года до 31 декабря 2020 года
dates = ("2010-01-01",  "2020-12-31")

# Выбираем посты в заданный промежуток
posts_by_date = postsData.filter(F.col("_CreationDate").between(*dates))
posts_by_date.show(10)

+-----------------+------------+--------------------+-----------+-------------+--------------------+--------------------+--------------+-------+--------------------+--------------------+----------------------+-----------------+-----------------+------------+---------+-----------+------+-----+------+----------+
|_AcceptedAnswerId|_AnswerCount|               _Body|_ClosedDate|_CommentCount| _CommunityOwnedDate|       _CreationDate|_FavoriteCount|    _Id|   _LastActivityDate|       _LastEditDate|_LastEditorDisplayName|_LastEditorUserId|_OwnerDisplayName|_OwnerUserId|_ParentId|_PostTypeId|_Score|_Tags|_Title|_ViewCount|
+-----------------+------------+--------------------+-----------+-------------+--------------------+--------------------+--------------+-------+--------------------+--------------------+----------------------+-----------------+-----------------+------------+---------+-----------+------+-----+------+----------+
|             NULL|        NULL|<p>No. (And more ...|       NULL

In [11]:
# 1. Скачиваем файл альтернативным способом (так как wget не работает в Windows)
csv_url = "https://git.ai.ssau.ru/tk/big_data/raw/branch/master/data/programming-languages.csv"
csv_filename = "programming-languages.csv"

if not os.path.exists(csv_filename):
    try:
        response = requests.get(csv_url)
        response.raise_for_status()  # Проверяем успешность запроса
        with open(csv_filename, 'wb') as f:
            f.write(response.content)
        print(f"Файл {csv_filename} успешно скачан")
    except Exception as e:
        print(f"Ошибка при скачивании файла: {e}")
else:
    print("Файл уже существует")

# 2. Получаем абсолютный путь к файлу
absolute_path = os.path.abspath(csv_filename)
print(f"Абсолютный путь к файлу: {absolute_path}")

# 3. Читаем файл с помощью Spark
try:
    languagesData = spark.read.format('csv') \
        .option('header', 'true') \
        .option("inferSchema", True) \
        .load(absolute_path) \
        .dropna()
    
    print("Данные успешно загружены!")
    languagesData.show(5)
except Exception as e:
    print(f"Ошибка при загрузке данных: {e}")
    print("Проверьте:")
    print(f"1. Существует ли файл: {os.path.exists(absolute_path)}")
    print(f"2. Размер файла: {os.path.getsize(absolute_path) if os.path.exists(absolute_path) else 0} байт")

Файл уже существует
Абсолютный путь к файлу: C:\Users\lena\Documents\Университет с 05-23\8 семестр\Биг дата\Lab_2\programming-languages.csv
Данные успешно загружены!
+----------+--------------------+
|      name|       wikipedia_url|
+----------+--------------------+
|   A# .NET|https://en.wikipe...|
|A# (Axiom)|https://en.wikipe...|
|A-0 System|https://en.wikipe...|
|        A+|https://en.wikipe...|
|       A++|https://en.wikipe...|
+----------+--------------------+
only showing top 5 rows



In [12]:
print("Схема")
languagesData.printSchema()

print("Первые 5 элементов")
languagesData.show(n = 5)

print("Описание датасета")
languagesData.describe().show()

print("Количество элементов")
print(languagesData.count())

Схема
root
 |-- name: string (nullable = true)
 |-- wikipedia_url: string (nullable = true)

Первые 5 элементов
+----------+--------------------+
|      name|       wikipedia_url|
+----------+--------------------+
|   A# .NET|https://en.wikipe...|
|A# (Axiom)|https://en.wikipe...|
|A-0 System|https://en.wikipe...|
|        A+|https://en.wikipe...|
|       A++|https://en.wikipe...|
+----------+--------------------+
only showing top 5 rows

Описание датасета
+-------+--------+--------------------+
|summary|    name|       wikipedia_url|
+-------+--------+--------------------+
|  count|     699|                 699|
|   mean|    NULL|                NULL|
| stddev|    NULL|                NULL|
|    min|@Formula|https://en.wikipe...|
|    max|xHarbour|https://en.wikipe...|
+-------+--------+--------------------+

Количество элементов
699


In [13]:
# Названия языков
language_names = [str(x[0]) for x in languagesData.collect()]
language_names

['A# .NET',
 'A# (Axiom)',
 'A-0 System',
 'A+',
 'A++',
 'ABAP',
 'ABC',
 'ABC ALGOL',
 'ABSET',
 'ABSYS',
 'ACC',
 'Accent',
 'Ace DASL',
 'ACL2',
 'ACT-III',
 'Action!',
 'ActionScript',
 'Ada',
 'Adenine',
 'Agda',
 'Agilent VEE',
 'Agora',
 'AIMMS',
 'Alef',
 'ALF',
 'ALGOL 58',
 'ALGOL 60',
 'ALGOL 68',
 'ALGOL W',
 'Alice',
 'Alma-0',
 'AmbientTalk',
 'Amiga E',
 'AMOS',
 'AMPL',
 'Apex (Salesforce.com)',
 'APL',
 "App Inventor for Android's visual block language",
 'AppleScript',
 'Arc',
 'ARexx',
 'Argus',
 'AspectJ',
 'Assembly language',
 'ATS',
 'Ateji PX',
 'AutoHotkey',
 'Autocoder',
 'AutoIt',
 'AutoLISP / Visual LISP',
 'Averest',
 'AWK',
 'Axum',
 'B',
 'Babbage',
 'Bash',
 'BASIC',
 'bc',
 'BCPL',
 'BeanShell',
 'Batch (Windows/Dos)',
 'Bertrand',
 'BETA',
 'Bigwig',
 'Bistro',
 'BitC',
 'BLISS',
 'Blockly',
 'BlooP',
 'Blue',
 'Boo',
 'Boomerang',
 'Bourne shell (including',
 'bash and',
 'ksh )',
 'BREW',
 'BPEL',
 'C',
 'C--',
 'C++ – ISO/IEC 14882',
 'C# – ISO/IEC

In [14]:
# Функция позволяет определить какой язык содержится в теге поста
def includes_name(x):
    tag = None
    for name in language_names:
        n = '<' + name.lower() + '>'
        if n in str(x._Tags).lower():
            tag = name
            break
    if tag is None:
        tag = 'No'

    return (x[6], tag)


In [15]:
# Преобразование DataFrame posts_by_date в RDD, применение функции includes_name и фильтрация результатов
posts_by_date_rdd = posts_by_date.rdd.map(includes_name).filter(lambda x: x[1] != 'No')

print(posts_by_date_rdd.take(1))  # посмотреть первую запись

[(datetime.datetime(2010, 9, 23, 16, 13, 59, 443000), 'Java')]


In [16]:
# Группировка данных по году и языку программирования, подсчет количества записей и сортировка по убыванию
posts_by_date_rdd_group = posts_by_date_rdd.keyBy(lambda row: (row[0].year, row[1])).aggregateByKey(0, lambda x, y: x + 1, lambda x1, x2: x1 + x2).sortBy(lambda x: x[1], ascending=False).collect()


In [17]:
# Создание списка years_list с годами от 2010 до 2020 в обратном порядке
years_list = [i for i in range(2010, 2021)][::-1]


In [18]:
# Формирование списка df_by_years с данными по топ-10 языкам программирования для каждого года из years_list
df_by_years = []
for year in years_list:
    df_by_years.extend([row for row in posts_by_date_rdd_group if row[0][0] == year][:10])

In [19]:
# Создание Row-шаблона row_template с полями 'Year', 'Language', 'Count'
row_template = Row('Year', 'Language', 'Count')

In [20]:
# Создание DataFrame result_df на основе данных из df_by_years и вывод его содержимого
result_df = spark.createDataFrame([row_template(*x, y) for x, y in df_by_years])
result_df.show()

+----+----------+-----+
|Year|  Language|Count|
+----+----------+-----+
|2019|    Python|  162|
|2019|JavaScript|  131|
|2019|      Java|   95|
|2019|       PHP|   59|
|2019|         R|   36|
|2019|         C|   14|
|2019|      Dart|    9|
|2019|    MATLAB|    9|
|2019|        Go|    9|
|2019|    Kotlin|    9|
|2018|    Python|  214|
|2018|JavaScript|  196|
|2018|      Java|  145|
|2018|       PHP|   99|
|2018|         R|   63|
|2018|         C|   24|
|2018|     Scala|   22|
|2018|TypeScript|   21|
|2018|PowerShell|   13|
|2018|      Bash|   12|
+----+----------+-----+
only showing top 20 rows



In [21]:
# Получившийся отчёт сохраняем в формате Apache Parquet
result_df.write.parquet("top_10_languages_between_2010_and_2020.parquet")

AnalysisException: [PATH_ALREADY_EXISTS] Path file:/C:/Users/lena/Documents/Университет с 05-23/8 семестр/Биг дата/Lab_2/top_10_languages_between_2010_and_2020.parquet already exists. Set mode as "overwrite" to overwrite the existing path.

In [ ]:

# Проверьте схему данных
result_df.printSchema()

# Проверьте, нет ли проблемных значений
result_df.show(5)

# Проверьте размер данных
print(f"Количество строк: {result_df.count()}")

In [ ]:
'''import os
from pyspark.sql import SparkSession

output_path = "top_10_languages_between_2010_and_2020.parquet"
absolute_path = os.path.abspath(output_path)

# Проверяем и очищаем путь
if os.path.exists(absolute_path):
    try:
        import shutil
        shutil.rmtree(absolute_path)
        print(f"Удалена существующая директория: {absolute_path}")
    except Exception as e:
        print(f"Ошибка при удалении директории: {e}")
        # Пробуем альтернативный путь
        output_path = "output/top_languages.parquet"
        absolute_path = os.path.abspath(output_path)
        os.makedirs(os.path.dirname(absolute_path), exist_ok=True)

try:
    (result_df.coalesce(1)  # Уменьшаем количество файлов
     .write
     .mode("overwrite")
     .parquet(absolute_path))
    print(f"Данные успешно сохранены в: {absolute_path}")
except Exception as e:
    print(f"Основной метод не сработал: {e}")
    
    # Альтернативный метод записи
    try:
        (result_df.write
         .format("parquet")
         .mode("overwrite")
         .save(absolute_path))
        print(f"Данные сохранены через альтернативный метод")
    except Exception as e2:
        print(f"Все методы записи не сработали: {e2}")

In [ ]:
'''
from pyspark.sql.functions import col, lower, regexp_extract, year, when, lit
from pyspark.sql import functions as F

# 1. Подготовка списка языков
language_patterns = [f"<{lang.lower()}>" for lang in language_names]

# 2. Создаем условие для поиска языков
condition = None
for pattern in language_patterns:
    current_cond = (F.lower(col("_Tags")).contains(pattern))
    if condition is None:
        condition = when(current_cond, lit(pattern[1:-1]))  # Убираем < и >
    else:
        condition = condition.when(current_cond, lit(pattern[1:-1]))

# Добавляем значение по умолчанию
language_cond = condition.otherwise("No")

# 3. Добавляем столбец с языком
posts_with_lang = posts_by_date.withColumn("language", language_cond)

# 4. Фильтруем записи с языком
filtered_posts = posts_with_lang.filter(col("language") != "No")

# 5. Добавляем год и группируем
result_df = (filtered_posts
             .withColumn("year", year(col("_CreationDate")))
             .groupBy("year", "language")
             .count())

# 6. Сортируем и показываем результат
sorted_df = result_df.orderBy("count", ascending=False)

# 7. Показываем результат (ограничиваем 20 строками)
display(sorted_df.limit(20))

# 8. Сохраняем в Parquet
sorted_df.write.mode("overwrite").parquet("top_languages_by_year.parquet")
'''